# DESaster Simulation Set Up Template

## Required Modules

In [1]:
import sys, random
desaster_path = "/Users/geomando/Dropbox/github/SeaGrantSimulation"
sys.path.append(desaster_path)
import simpy
from simpy import Resource, Container, FilterStore
from simpy.util import start_delayed
import pandas as pd
import numpy as np
from desaster.config import structural_damage_ratios
from desaster.config import acceleration_damage_ratios
from desaster.config import drift_damage_ratios

## Built Capital

In [2]:
class BuiltCapital(object):
    """Define top-level class for representing the attributes and methods
    of types of built capital.

    """
    def __init__(self, asset):
        """Run initial methods for defining built capital attributes.
        
        Keyword Arguments:
        asset -- A dataframe row with required built capital attributes.
        """
        self.setYearBuilt(asset)
        self.setValue(asset)
        self.setDamageState(asset)
        self.setInspection(asset)
    def setYearBuilt(self, asset):
        self.age = asset['Year Built']  # Year asset was built
    def setValue(self, asset):
        self.value = asset['Value']  # Value of the asset in $
    def setDamageState(self, asset):
        self.damage_state = asset['Damage State']  # HAZUS damage state
    def setInspection(self, asset):
        self.inspected = False  # Whether the asset has been inspected

In [3]:
class Building(BuiltCapital):
    """Define class that inherits from BuiltCapital() for representing the
    attributes and methods of types of buildings.
    """
    def __init__(self, building):
        """Run initial methods for defining building attributes.
        
        Keyword Arguments:
        building -- A dataframe row with required building attributes.
        """
        
        BuiltCapital.__init__(self, building)
        
        self.setOwner(building) # %***%
        self.setOccupant(building) # %***%
        self.setAddress(building)
        self.setOccupancy(building)
        self.setTenure(building)  # %***%
        self.setDamageValue(building)
       
    def setOwner(self, building):
        self.owner = building['Owner']  # Owner of building as Household() entity %***%
    def setOccupant(self, building):
        self.occupant = building['Occupant']  # Occupant of building as Household() entity %***%
    def setAddress(self, building):
        self.address = building['Address']  # Address of building
    def setOccupancy(self, building):
        self.occupancy = building['Occupancy']  # Occupancy type of building
    def setTenure(self, building):
        self.tenure = building['Tenure']  # Tenure of building: rent or own %***%
    def setBuildingArea(self, building):
        self.cost = building['Area']  # Floor area of building
    def setDamageValue(self, building):
        """Calculate damage value for building based on occupancy type and
        HAZUS damage state.

        Function uses three lookup tables (Table 15.2, 15.3, 15.4) from the HAZUS-MH earthquake model
        technical manual for structural damage, acceleration related damage,
        and for drift related damage, respectively. Estimated damage value for
        each type of damage is summed for total damage value.
        http://www.fema.gov/media-library/assets/documents/24609
        
        Keyword Arguments:
        structural_damage_ratios -- dataframe set in config.py
        acceleration_damage_ratios -- dataframe set in config.py
        drift_damage_ratios -- dataframe set in config.py
        """
        struct_repair_ratio = structural_damage_ratios.ix[building['Occupancy']][building['Damage State']] / 100.0
        accel_repair_ratio = acceleration_damage_ratios.ix[building['Occupancy']][building['Damage State']] / 100.0
        drift_repair_ratio = drift_damage_ratios.ix[building['Occupancy']][building['Damage State']] / 100.0
        self.damage_value = building['Value']*(struct_repair_ratio +
                                                accel_repair_ratio +
                                                drift_repair_ratio)

In [4]:
class Residence(Building):
    """Define class that inherits from Building() for representing the
    attributes and methods of types of residences.
    """
    def __init__(self, residence):
        """Run initial methods for defining residence attributes.

        Keyword Arguments:
        residence -- A dataframe row with required residence attributes.
        """
        
        Building.__init__(self, residence) # %***%
        
        self.setOccupancy(residence)
        self.setBedrooms(residence)
        self.setBathrooms(residence)
       

    def setOccupancy(self, residence):
        # Verify that residence dataframe has expected occupancy types
        if residence['Occupancy'] in ('Single Family Dwelling',
                            'Multi Family Dwelling', 'Mobile Home', 'Condo'):
            self.occupancy = residence['Occupancy']
        else:
            raise AttributeError(residence['Occupancy'])
    def setBedrooms(self, residence):
        self.bedrooms = residence['Bedrooms']  # Number of bedrooms in residence
    def setBathrooms(self, residence):
        self.bathrooms = residence['Bathrooms']  # Number of bathrooms in residence

In [5]:
def importHousingStock(simulation, stock_df):
    """Define, populate and return a SimPy FilterStore with Residence() objects to
    represent a vacant housing stock.
    
    Keyword Arguments:
    simulation -- Pointer to SimPy simulation environment.
    stock_df -- Dataframe with required attributes for each vacant home in
                the stock.
    """
    stock_fs = FilterStore(simulation)

    for i in stock_df.index:
        stock_fs.put(Residence(stock_df.loc[i]))

    return stock_fs

## Entities

In [6]:
class Household(object):
    """Define a Household() class to represent a group of persons that reside 
    together as a single analysis unit with attributes and methods.
    """
    def __init__(self, housing_stock, household_df, write_story = False):
        """Define household inputs and outputs attributes.
        Initiate household's story list string. 
        
        simulation -- Pointer to SimPy simulation environment.
        household_df -- Dataframe row w/ household input attributes.
        write_story -- Boolean indicating whether to track a households story.
        """
        
        # Household simulation inputs
        self.household = household_df  # Dataframe w/ household input attributes
#         self.name = household_df['Name']   # Name associated with household
        self.name = household_df['Occupant']   # Name associated with household %***%
        self.savings = household_df['Savings']  # Amount of household savings in $
        self.insurance = household_df['Insurance']  # Hazard-specific insurance coverage in $
        self.tenure_pref = household_df['Tenure Pref'] # Indicator of the household's preference between rent or own %***%
        self.tenure = household_df['Tenure'] # Indicator of the household's *actual* tenure between rent or own %***%
        self.occupancy_pref = household_df['Occupancy Pref'] # Indicator of the household's preference between occupancy types %***%
         
        # Household simulation outputs
        self.story = []  # The story of events for each household
        self.inspection_put = 0.0  # Time put request in for house inspection
        self.inspection_get = 0.0  # Time get  house inspection
        self.claim_put = 0.0  # Time put request in for insurance settlement
        self.claim_get = 0.0  # Time get insurance claim settled
        self.claim_payout = 0.0  # Amount of insurance claim payout
        self.assistance_put = 0.0  # Time put request in for FEMA assistance
        self.assistance_get = 0.0  # Time get FEMA assistance
        self.assistance_request = 0.0  # Amount of money requested from FEMA
        self.assistance_payout = 0.0  # Amount of assistance provided by FEMA
        self.money_to_rebuild = self.savings  # Total funds available to household to rebuild house
        self.home_put = 0.0  # Time put request in for house rebuild
        self.home_get = 0.0  # Time get house rebuild completed
        self.loan_put = 0.0  # Time put request for loan
        self.loan_get = 0.0  # Time get requested loan
        self.loan_amount = 0.0  # Amount of loan received
        self.permit_put = 0.0  # Time put request for building permit
        self.permit_get = 0.0  # Time get requested building permit
        self.home_search_start = 0.0  # Time started searching for a new home
        self.home_search_stop = 0.0  # Time found a new home
        self.money_search_start = 0.0  # Time that household started search for money
        self.money_search_stop = 0.0  # Time that household found rebuild money
        self.gave_up_money_search = False  # Whether household gave up search for money
        self.gave_up_home_search = False  # Whether household gave up search for home 
        
        # Initial method calls
        self.setStory(write_story)  # Start stories with non-disaster attributes
        self.setResidence(housing_stock, household_df)
    
    def setResidence(self, housing_stock, household_df):
        self.residence = Residence(household_df)
        housing_stock.put(self.residence)
        
    
    def setStory(self, write_story):
        """Initiate the household's story based on input attributes.
        
        Keyword Arguments:
        write_story -- Boolean indicating whether to track a households story.
        """
        if write_story == True:
            # Set story with non-disaster attributes.
            self.story.append(
            '{0} lives in a {1} bedroom {2} at {3} worth ${4:,.0f}. '.format(self.name, 
                                                            self.residence.bedrooms, 
                                                            self.residence.occupancy,
                                                            self.residence.address,
                                                            self.residence.value
                                                            )
            )

    def story_to_text(self): 
        """Join list of story strings into a single story string."""
        return ''.join(self.story)

In [7]:
def importHouseholds(housing_stock, households_df, write_story = False):
    """Return list of entities.Household() objects from dataframe containing
    data describing households.
    
    Keyword Arguments:
    simulation -- Pointer to SimPy simulation environment.
    household_df -- Dataframe row w/ household input attributes.
    write_story -- Boolean indicating whether to track a households story.
    """
    
    households = []

    # Population the simulation with households from the households dataframe
    for i in households_df.index:
        households.append(Household(housing_stock, households_df.iloc[i], write_story))
    
    return households

In [8]:
scenario_file = '../inputs/scenario_test_renters.xlsx'

# Create Pandas dataframe of attribute data for all households to be modeled in the simulation
households_df = pd.read_excel(scenario_file, sheetname='households')

# Create Pandas dataframe of attribute data for all vacant homes (housing stock) to be modeled in the simulation
housing_stock_df = pd.read_excel(scenario_file, sheetname='housing_stock')

# Set input data for all human capital types, as dict or Pandas Series
# .loc stuff is to convert the DataFrame to a Series ... data will function the same as a dictionary as well
human_cap_data = pd.read_excel(scenario_file, sheetname='human_capital', index_col=0).iloc[:,0]

# Set input data for all financial capital types, as dict or Pandas Series
financial_cap_data = pd.read_excel(scenario_file, sheetname='financial_capital', index_col=0).iloc[:,0]

In [9]:
households_df

,Occupant,Owner,Income,Savings,Insurance,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State,Tenure Pref,Tenure,Occupancy Pref
0,Alfred,Landlord,30000,10000,0,62 That St,Mobile Home,100,1,1,700,1920,100000,None,Own,Rent,Single Family Dwelling
1,Bruce,Bruce,100000,1000000,10000000,720 This Rd,Single Family Dwelling,100000,5,5,5000,1920,10000000,Slight,Own,Own,Single Family Dwelling
2,Selena,Landlord,10000,100,0,1001 Other Ave,Multi Family Dwelling,10,0,1,250,1960,10000,Complete,Rent,Rent,Multi Family Dwelling
3,Fish,Fish,50000,1000,550000,26000 Out There Lane,Single Family Dwelling,2000,4,2,2000,2010,800000,Moderate,Own,Own,Single Family Dwelling


In [10]:
housing_stock_df.head()

,Occupant,Owner,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State,Tenure
0,NaN,NaN,100 New Ave,Mobile Home,100,1,1,700,1920,99999,Complete,Own
1,NaN,NaN,101 New Ave,Single Family Dwelling,100000,6,5,5000,1920,9999,Complete,Own
2,NaN,NaN,102 New Ave,Multi Family Dwelling,10,0,1,250,1960,9999,Complete,Own
3,NaN,NaN,103 New Ave,Single Family Dwelling,2000,4,2,2000,2010,800000,Complete,Own
4,NaN,NaN,104 New Ave,Mobile Home,100,1,1,700,1920,100000,Complete,Own


## Simulation Initiation

__Set simulation environment__

In [11]:
simulation = simpy.Environment()

__Indicate whether want to keep track of the stories of each entity (household) in the simulation.__

In [12]:
write_story = False

__Populate the simulation with the input data specified above.__

In [13]:

vacant_housing = importHousingStock(simulation, housing_stock_df)

c

In [14]:
households = importHouseholds(occupied_housing, households_df, write_story)


In [17]:
households[0].residence

In [19]:
occupied_housing.items[0]

In [21]:
bob = occupied_housing.items[0]
bob

In [25]:
bill = bob
bill.age = 2000

In [26]:
bob.age

2000

In [ ]:
# query_res = occupied_housing.get(lambda getResidence: getResidence.occupant == 'Alfred')

In [ ]:
query_res

__Write a master process that combines process and functions from search, rebuild, and request modules.__

In [ ]:
occupied_housing.items

In [ ]:
def master_process(simulation, entity, write_story):
     
    print('occupied: ', len(occupied_housing.items))

    
    entity.residence = yield occupied_housing.get(lambda getResidence: getResidence.occupant == entity.name)
    
    print('vacant: ', len(vacant_housing.items))
    
    yield vacant_housing.put(entity.residence)

    print('vacant: ', len(vacant_housing.items))

In [ ]:
# Initiate a master process for each household to be modeled in the simulation
for i in range(len(households)):
    simulation.process(master_process(simulation, households[i], write_story))

In [ ]:
simulation.run()

In [ ]:
occupied_housing.items

In [ ]:
vacant_housing.items[]

In [ ]:
households[0].residence == vacant_housing.items[16]

In [ ]:
households[0].residence = []

## Outputs

__Household stories__

In [ ]:
households[0].residence.owner == households[0].residence.occupant

In [ ]:
households[1].story

In [ ]:
households[2].story

In [ ]:
households[3].story